In [5]:
## Library
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
from datetime import datetime
import time

In [6]:
def searchHTML(URL):
    #Request HTML & beautify it
    page = requests.get(URL)
    soup = bs(page.content)
    
    #Find right data
    results_numbers = soup.find_all("span", {'class' : 'sc-1ryi78w-0 cILyoi sc-16b9dsl-1 ZwupP u3ufsr-0 eQTRKC'})
    results_hash = soup.find_all("a", {'class' : 'sc-1r996ns-0 fLwyDF sc-1tbyx6t-1 kCGMTY iklhnl-0 eEewhk d53qjk-0 ctEFcK'})
    
    return results_numbers, results_hash

def makeFrame(hash, time, BTC, USD):
    dataF = {'Hash' : hash, 'Time' : time, 'BTC-value' : BTC, 'USD-value' : USD}
    frameUpdate = pd.DataFrame(dataF)
        
    return frameUpdate

def makeArrays(numbers, hash):
    only_time = [content.get_text() for content in numbers[0::3]]
    only_BTC = [content.get_text() for content in numbers[1::3]]
    only_USD = [content.get_text() for content in numbers[2::3]]
    only_hash = [content.get_text() for content in hash]
    
    return only_hash, only_time, only_BTC, only_USD

def fetchCurrent(URL):
    #HTML-doorzoeken en alles ophalen
    numbers, hash = searchHTML(URL)

    #Make arrays of only required values
    only_hash, only_time, only_BTC, only_USD = makeArrays(numbers, hash)

    #Append to existing
    frame = makeFrame(only_hash, only_time, only_BTC, only_USD)

    #Tells current time
    print()
    print(f"{datetime.now().hour}:{datetime.now().minute}")
    
    unique = frame['Time'].nunique()
    
    return frame, unique

In [10]:
#Eenmalig eventjes een lege dataframe maken om te kunnen appenden
dataF = {'Hash' : [], 'Time' : [], 'BTC-value' : [], 'USD-value' : []}
frame = pd.DataFrame(data=dataF)

In [15]:
while 1:
    URL = "https://www.blockchain.com/btc/unconfirmed-transactions"
    frame_new, unique = fetchCurrent(URL)
    appendedDF = frame.append(frame_new).drop_duplicates()
    frame = appendedDF



2:7

2:7

2:7

2:7

2:7


KeyboardInterrupt: 

In [16]:
print(appendedDF)

                                                 Hash   Time        BTC-value  \
0   0e61070427d410d472b41d113a6b0514466a042dabde9f...  01:06   0.00861650 BTC   
1   eefb42990bb7562016875f543e85957a98ee818a76285a...  01:06   0.00106044 BTC   
2   73473465ed7f5a0c4a5277c4d6cbb4f6256af81e601ba1...  01:06   0.04525660 BTC   
3   eba8c33fb2b4f8be544600f9ea5562a3e1147691bfbfc1...  01:06   0.09820926 BTC   
4   00fb7fcabacde7e7abf3fab1b4448f9d9e00e7ce24108f...  01:06   0.00375576 BTC   
..                                                ...    ...              ...   
43  70d041cd3705e31b7ff5d619a927c4f83f8b123a31d44d...  01:07   1.45692064 BTC   
44  af1f34e95f5cfead0eb31b6f9dac2d21190d4ec69ec456...  01:07   0.00002500 BTC   
45  e9b518742551995535c74a91342ae0471ab7e332b100a5...  01:07   9.54246107 BTC   
46  e1d36d13a2893c55454e24d0552c9fdfde7498897c6ff0...  01:07   0.00108580 BTC   
47  b6c4328a68b43208e16b5a64fd96db32b0b35887b5177f...  01:07  12.39664596 BTC   

      USD-value  
0       $

In [ ]:
print(frame['Time'])
#If == "00:26" voor cleaning bv.

0     00:26
1     00:26
2     00:26
3     00:26
4     00:26
      ...  
45    00:25
46    00:25
47    00:25
48    00:25
49    00:25
Name: Time, Length: 150, dtype: object
